In [ ]:
#1
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#2
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake -DBUILD_PYTHON=ON .. && make -j`nproc`
!mkdir openpose/images openpose/videos

In [ ]:
#3
!pip install --upgrade --no-cache-dir gdown

In [ ]:
#4
!pip install flask-ngrok
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok   
!ngrok authtoken YOUR-NGROK-AUTH-TOKEN-HERE

In [ ]:
#5
%cd /content/
!rm -rf clothes-virtual-try-on
!git clone https://github.com/practice404/clothes-virtual-try-on.git

In [ ]:
#6
!gdown --id 1ysEoAJNxou7RNuT9iKOxRhjVRNY5RLjx --output /content/clothes-virtual-try-on/cloth_segm_u2net_latest.pth --no-cookies

In [ ]:
#7
%cd /content/
!pip install ninja

In [ ]:
#8
!git clone https://github.com/PeikeLi/Self-Correction-Human-Parsing
%cd Self-Correction-Human-Parsing
!mkdir checkpoints

In [ ]:
#9
# downloading LIP dataset model
!gdown --id 1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH
!mv /content/Self-Correction-Human-Parsing/exp-schp-201908261155-lip.pth /content/Self-Correction-Human-Parsing/checkpoints/final.pth

In [ ]:
#10
%cd /content/

In [ ]:
#11
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX
conda install --channel defaults conda python=3.8 --yes
conda update --channel defaults --all --yes

In [ ]:
#12
import sys
_ = (sys.path
        .append("/usr/local/lib/python3.6/site-packages"))

In [ ]:
#13
!conda install --channel conda-forge featuretools --yes

In [ ]:
#14
!conda install -y pytorch torchvision cudatoolkit -c pytorch

In [ ]:
#15
!pip install opencv-python torchgeometry

In [ ]:
#16
def make_dir():
  os.system("cd /content/ && mkdir inputs && cd inputs && mkdir test && cd test && mkdir cloth cloth-mask image image-parse openpose-img openpose-json")

In [ ]:
#17
!nvidia-smi

In [ ]:
#18
from flask import Flask, request, send_file, jsonify
from flask_ngrok import run_with_ngrok
from PIL import Image
import base64
import io
from pdb import set_trace;
import os
from PIL import Image
import numpy as np
from collections import OrderedDict




app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
  return jsonify("Your API has deployed successfully");


@app.route("/api/transform", methods=['POST'])
def begin():
  make_dir()
  
  cloth = request.files['cloth']
  model = request.files['model']
  print("data recieved")

  cloth = Image.open(cloth.stream)
  model = Image.open(model.stream)

  cloth.save("/content/inputs/test/cloth/cloth.jpg")
  model.save("/content/inputs/test/image/model.jpg")

  print("image saved")
  os.system("python /content/clothes-virtual-try-on/run.py")

  # loading output
  op = os.listdir("/content/output")[0]
  op = Image.open(f"/content/output/{op}")
  buffer = io.BytesIO()
  op.save(buffer, 'png')
  buffer.seek(0)
  os.system("rm -rf /content/output/")
  return send_file(buffer, mimetype='image/gif')

  





  

In [ ]:
#19
app.run()